In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
import dash
from dash import dcc
from dash import html
import matplotlib.pyplot as plt

In [2]:
df_data = pd.read_excel('sp500_companies.xlsx', header=[0,1], index_col=0)

## Generating a data sample of 10 randomly drawn stocks

In [3]:
data_sample = df_data.sample(n=10, random_state=3)
#data_sample = df_data.sample(n=10, random_state=7)
#data_sample = df_data.sample(n=10, random_state=16)
#data_sample = df_data.sample(n=10, random_state=35)
#data_sample = df_data.sample(n=3, random_state=6)
data_sample

Environmental       Social       Governance       ESG   
     Environmental Score Social Score Governance Score ESG Score   
TYL             0.392972     0.212626         0.252625  0.858222  \
AIZ             0.390434     0.234261         0.234261  0.858956   
PRGO            0.383916     0.244034         0.231870  0.859821   
WAB             0.420916     0.235688         0.196920  0.853524   
CNC             0.387726     0.224629         0.246869  0.859224   
ORLY            0.390434     0.234261         0.234261  0.858956   
MHK             0.418801     0.225668         0.209747  0.854216   
DTE             0.410380     0.278386         0.163992  0.852758   
CRM             0.301939     0.261680         0.301939  0.865557   
WBA             0.378307     0.252205         0.229952  0.860464   

               Adj Close                                           
     2005-12-31 00:00:00 2006-12-31 00:00:00 2007-12-31 00:00:00   
TYL             8.780000           14.060000           12.890000  \
AIZ            31.569889           40.415440           49.329746   
PRGO           12.554116           14.724582           30.053989   
WAB            12.665672           14.322734           16.255201   
CNC             6.572500            6.142500            6.860000   
ORLY           32.009998           32.060001           32.430000   
MHK            86.980003           74.860001           74.400002   
DTE            18.405937           21.644058           20.533903   
CRM             8.012500            9.112500           15.672500   
WBA            29.072205           30.338011           25.379189   

                                                                  ...   
     2008-12-31 00:00:00 2009-12-31 00:00:00 2010-12-31 00:00:00  ...   
TYL            11.980000           19.910000           20.760000  ...  \
AIZ            22.462904           22.574562           30.047606  ...   
PRGO           27.903643           34.678921           55.395073  ...   
WAB            18.779510           19.316048           25.038275  ...   
CNC             4.927500            5.292500            6.335000  ...   
ORLY           30.740000           38.119999           60.419998  ...   
MHK            42.970001           47.599998           56.759998  ...   
DTE            17.571611           22.915539           24.965193  ...   
CRM             8.002500           18.442499           33.000000  ...   
WBA            16.660372           25.195103           27.250443  ...   

                                                                   
     2013-12-31 00:00:00 2014-12-31 00:00:00 2015-12-31 00:00:00   
TYL           102.129997          109.440002          174.320007  \
AIZ            54.971760           57.597458           69.083206   
PRGO          135.522858          148.036011          128.530365   
WAB            70.718071           82.940140           68.100258   
CNC            14.737500           25.962500           32.904999   
ORLY          128.710007          192.619995          253.419998   
MHK           148.899994          155.360001          189.389999   
DTE            41.546703           56.004925           53.937138   
CRM            55.189999           59.310001           78.400002   
WBA            43.263714           58.551399           66.523758   

                                                                   
     2016-12-31 00:00:00 2017-12-31 00:00:00 2018-12-31 00:00:00   
TYL           142.770004          177.050003          185.820007  \
AIZ            81.651482           90.619316           82.300964   
PRGO           74.365349           78.517807           35.272038   
WAB            79.893112           78.805153           68.356728   
CNC            28.254999           50.439999           57.650002   
ORLY          278.410004          240.539993          344.329987   
MHK           199.679993          275.899994          116.959999   
DTE            68.476349           78.467461           81.768974   
CRM

## Calculating log returns of the data sample

In [4]:
from log_return import log_returns
log_re = log_returns(data_sample['Adj Close']).replace([np.inf, -np.inf], np.nan).dropna(axis=1).T

In [5]:
log_re.mean().mean()

0.11945243383431788

In [6]:
log_re.std().mean()

0.26265868884621557

In [7]:
srs = log_re.mean() / log_re.std()
srs.mean()

0.5085683925565367

In [8]:
log_re.skew().mean()

-0.38805475007966933

## Fetching ESG scores from the data sample for later use

In [9]:
from fetch_score import get_scores
esg_np, e_np, s_np, g_np = get_scores(data_sample)

## Calculating the Covariance matrix, inverse covariance matrix and mean return for the stocks

In [10]:
from covariance import cov_matrix
from mean_ret import mean_return

cov_var = cov_matrix(log_re)
inv_cov = np.linalg.inv(cov_var)
mean_re = mean_return(log_re)

## Finding two optimal portfolios with different expected return

In [11]:
from ones import vector_ones
from mvp import mvp_func
from weights_mvp import mvp_weights

labels = data_sample.index
vector_one = vector_ones(labels)
a, b, c, d = mvp_func(inv_cov, mean_re, vector_one)
w1, w2 = mvp_weights(a, b, c, d, inv_cov, vector_one, mean_re, 0.10, 0.26)

## Two optimal portfolios are now obtained. Thus, we can use the two fund theorem to find the weights for the entire efficient frontier. Hereby, we also compute the portfolios expected returns, volatility and weighted avereage esg score

In [12]:
from alpha import alpha
from two_fun_theorem import w3

alpha_list = alpha([-3.5])
ef_weights, r, vol, esg_wa = w3(w1, w2, alpha_list, mean_re, cov_var, esg_np)
len(ef_weights)

601

In [13]:
esg_wa

array([0.86550064, 0.8654811 , 0.86546156, 0.86544202, 0.86542248,
       0.86540295, 0.86538341, 0.86536387, 0.86534433, 0.86532479,
       0.86530525, 0.86528572, 0.86526618, 0.86524664, 0.8652271 ,
       0.86520756, 0.86518803, 0.86516849, 0.86514895, 0.86512941,
       0.86510987, 0.86509033, 0.8650708 , 0.86505126, 0.86503172,
       0.86501218, 0.86499264, 0.8649731 , 0.86495357, 0.86493403,
       0.86491449, 0.86489495, 0.86487541, 0.86485588, 0.86483634,
       0.8648168 , 0.86479726, 0.86477772, 0.86475818, 0.86473865,
       0.86471911, 0.86469957, 0.86468003, 0.86466049, 0.86464096,
       0.86462142, 0.86460188, 0.86458234, 0.8645628 , 0.86454326,
       0.86452373, 0.86450419, 0.86448465, 0.86446511, 0.86444557,
       0.86442603, 0.8644065 , 0.86438696, 0.86436742, 0.86434788,
       0.86432834, 0.86430881, 0.86428927, 0.86426973, 0.86425019,
       0.86423065, 0.86421111, 0.86419158, 0.86417204, 0.8641525 ,
       0.86413296, 0.86411342, 0.86409389, 0.86407435, 0.86405

### Tickerlist for the plot

In [14]:
tickers_list = []
for i in range(len(ef_weights)):
    #assets = np.random.choice(list(labels), len(labels), replace=False)
    assets = list(labels)
    tickers_list.append(assets)
sr_t_list = list(labels)

## Obtaining the global MVP. Furthermore, we calculate the expected return, volatility and WA ESG score for the portfolio

In [15]:
from global_mvp import global_mvp
mvp_weight, mvp_er, mvp_vol, mvp_esg_score, mvp_e_score, mvp_s_score, mvp_g_score = global_mvp(ef_weights, vol, mean_re, cov_var, esg_np, e_np, s_np, g_np)

In [16]:
from esg_indicators import indicators
E_0, V_0 = indicators(vector_one, inv_cov, mean_re, r, esg_np, 0.863)

Variable outputs used from the paper
------------------------------------
the value of delta 0 is: 0.8554873027281915

-------------------------------------
Is delta 1 larger than zero: True
-------------------------------------
The value of delta 1 is: 0.012211382718217876

-------------------------------------
The value of Lambda 0 is: 1.5802899804393653

-------------------------------------
The value of gamma 0 is: 0.1609071201470904

-------------------------------------
The value of gamma 1 is: 0.717947437369225

-------------------------------------
The value of gamma 2 is: 0.030677240381956218

--------------------------------------
Is MVP ESG rating higher than ESG threshold: False
--------------------------------------
the ESG score of the MVP is: 0.8571218802488549

--------------------------------------
the WA ESG score for traditional investor portfolios is: [0.86550064 0.8654811  0.86546156 0.86544202 0.86542248 0.86540295
 0.86538341 0.86536387 0.86534433 0.86532479 0.86

In [17]:
from esg_investor_weights import minimize_func
list_target_returns = np.arange(r.min(), r.max(), 0.0016).tolist()
esg_investor_weights, esg_portfolio_returns, esg_portfolio_vol, esg_investor_wa  = minimize_func(mean_re, cov_var, esg_np, 0.863, list_target_returns, mvp_weight)

## Sharpe Ratio for the traditional investor and ESG investor

In [18]:
from max_sr import sharpe_ratio1
sr, w_opt, sharpe_exp, sharpe_vol, sr_esg_score, sr_e_score, sr_s_score, sr_g_score, sr_esg, w_opt_esg, sharpe_exp_esg, sharpe_vol_esg, sr_esg_esg_score, sr_esg_e_score, sr_esg_s_score, sr_esg_g_score = sharpe_ratio1(0.03, mean_re, cov_var, ef_weights, r, vol, esg_investor_weights, esg_portfolio_returns, esg_portfolio_vol, esg_np, e_np, s_np, g_np, mvp_weight, log_re.columns, 0.863)

## Sortino Ratio for the traditional investor and ESG investor

In [21]:
from sortino import optimize_sortino_ratio
optimal_weights, optimal_sortino_ratio, sortino_exp, sortino_vol, sortino_score, sortino_e_score, sortino_s_score, sortino_g_score, optimal_weights_esg, optimal_sortino_ratio_esg, sortino_exp_esg, sortino_vol_esg, sortino_score_esg, sortino_e_score_esg, sortino_s_score_esg, sortino_g_score_esg = optimize_sortino_ratio(log_re, 0.03, labels, mvp_weight, mean_re, cov_var, esg_np, e_np, s_np, g_np, 0.863)

In [51]:
optimal_sortino_ratio_esg

4.048213860873771

In [52]:
sortino_exp_esg

0.6500769109859783

In [59]:
sortino_vol_esg

0.6870847831182184

In [54]:
sortino_score_esg

0.8630000007874644

In [56]:
sortino_e_score_esg

0.319392731136443

In [57]:
sortino_s_score_esg

0.20578259322348735

In [58]:
sortino_g_score_esg

0.33782467642753433

## Plotting the Efficient Frontiers

In [24]:
from plot_ef import ef_plot
plot = ef_plot(tickers_list, sr_t_list, 
               ef_weights, vol, r, esg_wa, 
               esg_investor_weights, esg_portfolio_vol, esg_portfolio_returns, esg_investor_wa, 
               w_opt, sharpe_vol, sharpe_exp, sr, sr_esg_score,
               w_opt_esg, sharpe_vol_esg, sharpe_exp_esg, sr_esg, sr_esg_esg_score,
               mvp_weight, mvp_vol, mvp_er, mvp_esg_score, 
               optimal_weights, sortino_vol, sortino_exp, optimal_sortino_ratio, sortino_score, 
               optimal_weights_esg, sortino_vol_esg, sortino_exp_esg, optimal_sortino_ratio_esg, sortino_score_esg, E_0, V_0,
               ratio='Sortino Ratio')

plot.show()

In [25]:
benchmark = pd.read_excel("sp500_bench.xlsx", header=[0], index_col=0)
benchmark

2010-12-31   2011-12-31   2012-12-31   2013-12-31   2014-12-31   
sp500  1257.640015  1257.599976  1426.189941  1848.359985  2058.899902  \

        2015-12-31   2016-12-31   2017-12-31   2018-12-31   2019-12-31   
sp500  2043.939941  2238.830078  2673.610107  2506.850098  3230.780029  \

        2020-12-31   2021-12-31  2022-12-31  
sp500  3756.070068  4766.180176      3839.5

In [27]:
from log_return import log_returns
log_market_returns = log_returns(benchmark).replace([np.inf, -np.inf], np.nan).dropna(axis=1).T

In [28]:
from treynor import treynor
jensens_alpha, beta,timing = treynor(12, log_re, w_opt, log_market_returns, 0.03)

In [29]:
alpha_esg, beta_esg, timing_esg = treynor(12, log_re, w_opt_esg, log_market_returns, 0.03)

In [32]:
alpha_esg

0.27800728027255994

In [30]:
alpha_esg_sortino, beta_esg_sortino, timing_esg_sortino = treynor(12, log_re, optimal_weights, log_market_returns, 0.03)

In [31]:
alpha_esg_sortino

0.9892550099904005